In [1]:
# Imports

import os
import torch, torchvision
import numpy as np
from torchvision import transforms, utils
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import glob

In [2]:
# checking if gpu is available
torch.cuda.is_available()

True

In [3]:
import warnings
warnings.filterwarnings("ignore")

## Making Dataset

<b>Dataset info</b> :<br><t><p>
    ``ASL dataset created by B. Kang et al is used. It is a collection of 31,000 images, 1000 images for each of the 31 classes. These gestures are recorded for a total of five subjects. The gestures include numerals 1- 9 and alphabets A-Z except ‘J’ and ‘Z’, because these require movements of hand and thus cannot be captured in the form of an image. Some of the gestures are very similar, (0/o) , (V/2) and (W/6). These are classified by context or meaning.``

In [4]:
def find_classes(dir):   # Finds the class folders in a dataset, dir (string): Root directory path.

        classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

In [5]:
class ASLDataset(Dataset):
    """ American Sign Language Dataset """
    
    def __init__(self, image_paths, classes, class_to_idx, path):
        
        self.image_paths = image_paths
        self.transforms = transforms.ToTensor()
        self.class_to_idx = class_to_idx
        classes, class_to_idx = find_classes(path)
       
    
    def __getitem__(self, index):
        
        image = Image.open(self.image_paths[index])
        transform_image = image.convert("L")
        transform_image = self.transforms(transform_image)
        
        class_to_idx = self.class_to_idx
        
        return transform_image, class_to_idx, self.image_paths[index]
        
    def __len__(self): 

        return len(self.image_paths)
    

### Setting data splits

In [6]:
path = '/home/nayugartx8/Desktop/Aayush_work/Sign_language_model/Train/*/*.*'
folder_data = glob.glob(path)
np.savetxt('distribution_class.csv', np.c_[folder_data], fmt=['%s'], comments='', delimiter = ",")

In [7]:
len_data = len(folder_data)
len_data

87000

Splits : 60 % Train, 20 % validation, 10% test

In [8]:
split_1 = int(0.6 * len(folder_data))
split_2 = int(0.8 * len(folder_data))

split_1, split_2

(52200, 69600)

In [9]:
folder_data.sort()

train_image_paths = folder_data[:split_1]
print("count of train images is: ", len(train_image_paths)) 
np.savetxt('im_training_path_1.csv', np.c_[train_image_paths], fmt=['%s'], comments='', delimiter = ",") 

count of train images is:  52200


In [10]:
valid_image_paths = folder_data[split_1:split_2]
print("count of validation images is: ", len(valid_image_paths)) 
np.savetxt('im_validation_path_1.csv', np.c_[valid_image_paths], fmt=['%s'], comments='', delimiter = ",") 

count of validation images is:  17400


In [11]:
test_image_paths = folder_data[split_2:]
print("count of test images is: ", len(test_image_paths)) 
np.savetxt('im_testing_path_1.csv', np.c_[test_image_paths], fmt=['%s'], comments='', delimiter = ",")

count of test images is:  17400


In [12]:
# Getting Label and Label map

classes = sorted([subfolder for subfolder in os.listdir('./Train')])
classes_to_idx = {i: j for j, i in enumerate(classes)}
classes, classes_to_idx

(['A',
  'B',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'J',
  'K',
  'L',
  'M',
  'N',
  'O',
  'P',
  'Q',
  'R',
  'S',
  'T',
  'U',
  'V',
  'W',
  'X',
  'Y',
  'Z',
  'del',
  'nothing',
  'space'],
 {'A': 0,
  'B': 1,
  'C': 2,
  'D': 3,
  'E': 4,
  'F': 5,
  'G': 6,
  'H': 7,
  'I': 8,
  'J': 9,
  'K': 10,
  'L': 11,
  'M': 12,
  'N': 13,
  'O': 14,
  'P': 15,
  'Q': 16,
  'R': 17,
  'S': 18,
  'T': 19,
  'U': 20,
  'V': 21,
  'W': 22,
  'X': 23,
  'Y': 24,
  'Z': 25,
  'del': 26,
  'nothing': 27,
  'space': 28})

In [14]:
paths = '/home/nayugartx8/Desktop/Aayush_work/Sign_language_model/Train/'
train_dataset = ASLDataset(train_image_paths, classes_to_idx, classes, paths)
train_loader = torch.utils.data.DataLoader(train_dataset)

In [25]:
valid_dataset = ASLDataset(valid_image_paths,  classes_to_idx, classes,paths)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=0)


test_dataset = ASLDataset(test_image_paths, classes_to_idx, classes, paths)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0) 

In [27]:
dataLoaders = {
        'train': train_loader,
        'valid': valid_loader,
         'test': test_loader,
        }